In [1]:
# Libraries importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline
from collections import Counter
from sklearn.metrics import accuracy_score
import pymorphy2
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle


In [10]:
test_data = pd.read_csv('data/test.csv')
data = pd.read_csv('data/train.csv')

In [11]:
def count_vowels(s):
    count = 0
    vowels = set("аеёиоуыэюя")
    for letter in s:
        if letter in vowels:
            count += 1
    return count

morph = pymorphy2.MorphAnalyzer()

data['upper']=data['Word'].apply(lambda x: 1 if x[0].isupper() else 0)
data['all_upper']=data['Word'].apply(lambda x: 1 if x.isupper() else 0)

data['Word'] = data['Word'].apply(lambda x: morph.parse(x)[0].normal_form)

c = Counter(data[data['Label']==1]['Word'].apply(lambda x: x[-3:].lower()))
surname_endings_list=[]
for word, count in c.most_common(200):
    surname_endings_list.append(word)
    
c = Counter(data[data['Label']==1]['Word'].apply(lambda x: x[-2:].lower()))
surname_endings_list_2=[]
for word, count in c.most_common(200):
    surname_endings_list_2.append(word)

data['letters_count']=data['Word'].apply(lambda x: len(x))

data['vowels_count'] = data['Word'].apply(lambda x: count_vowels(x))
data['consonants_count'] = data['letters_count'] - data['vowels_count']

    
data['surname_ending']=data['Word'].apply(lambda x: 1 if x[-3:] in surname_endings_list else 0)
data['surname_ending_2']=data['Word'].apply(lambda x: 1 if x[-2:] in surname_endings_list_2 else 0)


In [12]:
d = data.drop(['Word', 'Label'], axis=1)
labels = data.iloc[0:,1:2]

train_words, test_words,train_labels, test_labels = train_test_split(d,labels, 
                                                                         train_size=0.8, random_state=0)

/home/alena/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [19]:
test_data = pd.read_csv('data/test.csv')

test_data['upper']=test_data['\ufeffWord'].apply(lambda x: 1 if x[0].isupper() else 0)
test_data['all_upper']=test_data['\ufeffWord'].apply(lambda x: 1 if x.isupper() else 0)

test_data['\ufeffWord'] = test_data['\ufeffWord'].apply(lambda x: morph.parse(x)[0].normal_form)

test_data['letters_count']=test_data['\ufeffWord'].apply(lambda x: len(x))

test_data['vowels_count'] = test_data['\ufeffWord'].apply(lambda x: count_vowels(x))
test_data['consonants_count'] = test_data['letters_count'] - test_data['vowels_count']

    
test_data['surname_ending']=test_data['\ufeffWord'].apply(lambda x: 1 if x[-3:] in surname_endings_list else 0)
test_data['surname_ending_2']=test_data['\ufeffWord'].apply(lambda x: 1 if x[-2:] in surname_endings_list_2 else 0)

t_data = test_data

t_data = t_data.drop(['\ufeffWord'], axis=1)

In [14]:
knn = KNeighborsClassifier()

#i = 6
#train_words, test_words,train_labels, test_labels = train_test_split(d.iloc[i * 10140:(i + 1) * 10140], labels.iloc[i * 10140:(i + 1) * 10140], 
#                                                                         train_size=0.8, random_state=0)

knn.fit(train_words, train_labels.values.ravel())
train_labels_predict = knn.predict(train_words)
test_labels_predict = knn.predict(test_words)


print(accuracy_score(train_labels,train_labels_predict))
print(accuracy_score(test_labels,test_labels_predict))

0.8936222665976382
0.8911349965486638


In [22]:
def func(row):
  return 1 if row['\ufeffWord'] in surnames else row['Label']

predictions=knn.predict_proba(t_data)
test_data['Label'] = predictions[:,1]
surnames=data[data['Label']==1]['Word'].tolist()
test_data['Label']=test_data.apply(func, axis = 1)
prediction = pd.DataFrame(test_data).to_csv('prediction.csv')